# Урок 4

# Системы управления базами данных MongoDB и SQLite в Python

## На этом уроке 

- Разберём основные принципы работы с реляционными и нереляционными базами данных.
- Рассмотрим основные операции и методы для формирования запросов. 
- Научимся работать с данными внутри баз.

## SQL и noSQL

СУБД (DBMS, Database Management System) — система управления базами данных. Это набор команд, прикладных и инфраструктурных приложений, а также библиотек, которые позволяют управлять и обслуживать базу данных. По сути, как API для программиста. 

База данных (БД) — структурированный массив данных. Он может располагаться как на жёстком диске, так и в оперативной памяти. Есть множество типов СУБД, но сейчас актуальны два — реляционные (SQL) и нереляционные (noSQL).

### SQL 

Реляционные БД основаны на реляционной модели данных. Базы данных SQL используют язык структурированных запросов для определения данных и управления ими. SQL — универсальный и широко используемый вариант, который хорошо подходит для сложных запросов и гарантирует безопасность. Однако он может носить ограничительный характер, поскольку требует использования предопределённых схем для определения структуры данных перед работой с ними. У всех данных должна быть одинаковая структура, так что предварительная подготовка может отнять немало усилий. 

### NoSQL

Базы данных NoSQL, наоборот, имеют динамические схемы для неструктурированных данных. Данные здесь хранятся разными способами: они могут быть ориентированы на столбцы или на документы, основаны на графах или организованы как хранилище KeyValue. Такая гибкость означает, что:

- вы можете создавать документы без предварительного определения их структуры;
- каждый документ может иметь свою уникальную структуру;
- синтаксис может варьироваться от базы данных к базе данных;
- вы можете в любой момент добавлять новые поля.

Всего есть 4 типа баз данных NoSQL:

- __Ключ-значение__ (Redis, Berkeley DB). Хранилища типа key-value — простейшие БД, чаще всего это in-memory базы данных (то есть они хранятся и работают в оперативной памяти сервера). Такой БД не требуется ни схемы, ни связи.
- __Документоориентированные__ (MongoDB, CouchDB). В таких БД данные хранятся в виде документов и имеют сложную иерархическую (древовидную) структуру, между элементами есть связи.
- __Графовые__ (Giraph, Neo4j). Такие БД позволяют хорошо реализовать семантические паутины (например, социальные сети), в подобных задачах они более производительны;
- __BigTable__ (HBase, Cassandra). В них данные представлены в виде разреженной матрицы. Эти БД похожи на документоориентированные.

### Документная модель данных

Модель данных MongDB — документоориентированная. Для тех, кто не знаком с идеей документа в контексте баз данных, продемонстрировать её проще всего на примере.

```python
{
    _id: ObjectID("4bd9e8e17cefd644108961bb"),   #Поле _id - первичный ключ
    title: "Adventure in Databases",
    url: "http://example.com/databases.txt",
    author: "msmith",
    vote_count: 20,
    tags: ['databases', 'mongodb', 'indexing'],  #Теги хранятся в виде массива строк
    image:{                                      #Атрибут указывает на другой элемент
        url: "http://example.com/db.jpg",
        caption:"",
        type: "jpg",
        size: 75381,
        data: "Binary"
    },
    comments: 
    [                     #Комментарии хранятся в виде массива объектов, 
        {                             #представляющих ещё один комментарий
            user: "bjones",
            text: "Interesting article!"
        },
        {
            user: "blogger",
            text: "Another related article is at http://example.com/db/db.txt"
        }
    ]
}
```

Это пример документа, представляющего статью на социальном новостном сайте. Как видите, документ — это набор, состоящий из имён и значений свойств. Значение может быть представлено простым типом: например, строки, числа и даты. Но может быть также последовательностью и даже другим документом. С помощью таких конструкций можно представлять весьма сложные структуры данных. Так, в нашем примере имеется свойство tags — список, в котором хранятся ассоциированные со статьёй теги. Но ещё интереснее свойство comments, которое ссылается на список документов, содержащих комментарии.

Сравним это с представлением тех же данных в стандартной реляционной базе:

<img src="https://i.ibb.co/TWXDyVt/1.jpg"  width = 1000/>

Здесь изображён типичный реляционный аналог. Поскольку таблицы, по сути, плоские, для представления связей типа «один-ко-многим» нужно несколько таблиц. Мы начинаем с таблицы posts, в которой хранится основная информация о каждой статье. Затем создаём ещё три таблицы, каждая из которых содержит поле post_id, ссылающееся на исходную статью. 

Вероятно, вы обратили внимание, что документы не только позволяют представлять данные со сложной структурой, но и не нуждаются в заранее определённой схеме. В реляционной базе данных строки хранятся в таблице. У каждой таблицы строго определённая схема, описывающая, какие столбцы и типы данных допустимы. Если окажется, что нужно добавить ещё одно поле, таблицу придётся менять.

В MongoDB документы группируются в коллекции — контейнеры, не налагающие на данные какую-либо схему. Теоретически у каждого входящего в коллекцию документа может быть своя структура, но на практике документы в одной коллекции похожи друг на друга. Например, у всех документов в коллекции posts есть поля title, tags, comments и так далее.

### Произвольные запросы

Рассмотрим принцип построения запросов в MongoDB на простом примере статей и комментариев. Пусть нужно найти все статьи, помеченные тегом politics, за которые проголосовало более 10 посетителей. SQL-запрос для решения этой задачи выглядел бы так:

```sql
SELECT * FROM posts
INNER JOIN posts_tags ON posts.id = posts_tags.post_id INNER JOIN tags ON posts_tags.tag_id == tags.id
WHERE tags.text = 'politics' AND posts.vote_count > 10;
```

Эквивалентный запрос в MongoDB формулируется путём задания документа-образца. Условие «больше» обозначается специальным ключом `$gt`.

```python
db.posts.find(('tags': 'politics', 'vote_count': {'$gt': 10}});
```

Стоит отметить, что в этих запросах предполагаются разные модели данных. SQL-запрос опирается на строго нормализованную модель, в которой статьи и теги хранятся в разных таблицах,  поэтому мы объединяем таблицы при помощи JOIN, тогда как в запросе для MongoDB считается, что теги хранятся внутри документа, описывающего статью.

## Работа с MongoDB

## Установка MongoDB

MongoDB — кроссплатформенная СУБД. Чтобы скачать дистрибутив, нужно перейти по ссылке [Download Center: Community Server](https://www.mongodb.com/try/download/community), выбрать свою ОС и нажать кнопку Download. А затем установить скачанный архив, следуя подсказкам мастера установки. По желанию место установки можно поменять, например (C:\mongodb\bin)

Инструкция по установке — [Install MongoDB Community Edition](https://www.mongodb.com/docs/manual/administration/install-community/). 

## Содержимое пакета MongoDB

Если после установки мы откроем папку bin в распакованном архиве (C:\mongodb\bin), то сможем найти там кучу приложений, которые выполняют определённую роль. Вкратце рассмотрим их.

- __mongo__ (в последних версиях убрали, можно установить отдельно [__mongosh__](https://www.mongodb.com/try/download/shell)) — консольный интерфейс для взаимодействия с базами данных, своего рода консольный клиент;
- __mongod__ — сервер баз данных MongoDB, обрабатывает запросы, управляет форматом данных и выполняет различные операции в фоновом режиме по управлению базами данных;
- __mongos__ — служба маршрутизации MongoDB, которая помогает обрабатывать запросы и определять местоположение данных в кластере MongoDB.

### Создание каталога для БД и запуск MongoDB на Windows

__!!! ВАЖНО !!! В последних версия mongoDB данных шаг можно пропустить. Единственное, если хотите работать с консолью - придется настроить пространство имен, подробнее об этом на уроке__

После установки надо создать на жёстком диске каталог, в котором будут находиться базы данных MongoDB.
В ОС Windows по умолчанию MongoDB хранит базы данных по пути C:\data\db, поэтому, если вы используете Windows, вам надо создать соответствующий каталог.

Если возникла необходимость использовать какой-то другой путь к файлам, его можно передать при запуске MongoDB во флаге --dbpath.

После создания каталога для хранения БД можно запустить сервер MongoDB. Сервер представляет приложение mongod, которое находится в папке bin. Для этого запустим терминал/командную строку и там введём соответствующие команды. Для ОС Windows это будет выглядеть так:


<img src="https://i.ibb.co/wShrXQ3/2.jpg"/>

Командная строка отобразит нам ряд служебной информации, например, что сервер запускается на localhost на порту 27017. После удачного запуска сервера мы сможем производить операции с БД через оболочку mongo. Эта оболочка представляет файл mongo.exe, который располагается в выше рассмотренной папке установки. Запустим этот файл:

<img src="https://i.ibb.co/RNFPR0g/3.jpg"/>

Это консольная оболочка для взаимодействия с сервером, через которую можно управлять данными. Второй строкой эта оболочка говорит о подключении к серверу mongod.

### Запуск на MacOS

Для запуска сервера MongoDB выполним в терминале команду:

```
brew services start mongodb-community@5.0
```

Для отключения сервера — команду:

```
brew services stop mongodb-community@5.0
```

Итак, после удачного запуска сервера мы сможем производить операции с БД через оболочку mongosh. Запустим её, выполнив в терминале команду mongosh.

## Работа с MongoDB из консоли

Чтобы узнать, какие базы данных у вас есть:

```python
> show dbs
admin              0.000GB
config             0.000GB
local              0.000GB
test               0.000GB
```

Чтобы подключиться к текущей базе данных или создать новую, используем команду use:

```python
> use newdb
switched to db newdb
```

__Важно!__ В MongoDB используется принцип экономии: наша СУБД не будет создавать базу данных до тех пор, пока она пустая.

Теперь произведём какие-либо простейшие действия. Введём в mongo последовательно следующие команды и после каждой нажмём enter:

```python
> use mydb
> db.users.insertOne( { name: "Tom" } )
> db.users.find()
```

Первая команда `use mydb` устанавливает в качестве используемой базу данных mydb. Даже если такой БД нет, она создаётся автоматически. И далее db будет представлять текущую базу данных — то есть базу данных mydb. После db идёт `users` — это коллекция, в которую затем мы добавляем новый объект. Если в SQL нам надо создавать таблицы заранее, то коллекции MongoDB создаёт самостоятельно при их отсутствии.

С помощью метода `db.users.insertOne()` в коллекцию users базы данных mydb добавляется объект `{name: "Tom" }`. Описание добавляемого объекта определяется в формате, с которым вы, возможно, знакомы, если имели дело с форматом JSON. То есть в данном случае у объекта определён один ключ "name", которому сопоставляется значение `"Tom"`. То есть мы добавляем пользователя с именем Tom.

Если объект был успешно добавлен, то консоль выведет результат операции, в частности, идентификатор добавленного объекта.
Третья команда `db.users.find()` выводит на экран все объекты из БД mydb.

Если в базе данных не остаётся ни одной коллекции, то она удаляется из общего списка. Удалим только что созданную коллекцию из БД:

```python
> db.users.drop()
true
```

Проверим снова список существующих баз данных:

```python
> show dbs
admin              0.000GB
config             0.000GB
local              0.000GB
test               0.000GB
```

Как видим, база данных mydb также удалилась.

### Операции CRUD

CRUD-операции — это создание (Create), чтение (Read), обновление (Update), Удаление (Delete).

Общий синтаксис операции:

<img src="https://i.ibb.co/VNPDtfQ/4.jpg"/>

### Работа с MongoDB в Python

Прежде чем начать, необходимо установить модуль PyMongo:





In [ ]:
#!pip install pymongo

Первый шаг при работе с PyMongo — это подключение модуля __MongoClient__:

In [ ]:
from pymongo import MongoClient

Затем создадим массив, который потом вставим в коллекцию

In [ ]:
cars_dict = [
    {'name': 'Audi', 'price': 52642},
    {'name': 'Mercedes', 'price': 57127},
    {'name': 'Skoda', 'price': 9000},
    {'name': 'Volvo', 'price': 29000},
    {'name': 'Bentley', 'price': 350000},
    {'name': 'Citroen', 'price': 21000},
    {'name': 'Hummer', 'price': 41400},
    {'name': 'Volkswagen', 'price': 21600}
]

Создадим клиента для работы с монгой, передав ему имя хоста и порт:

In [ ]:
client = MongoClient('mongodb://127.0.0.1:27017/')

Посмотрим список баз данных:

In [ ]:
print(client.list_database_names()) 

['admin', 'config', 'local']


Создаём ссылку на базу данных testdb:

In [ ]:
db = client.testdb

С помощью метода `insеrt_many()` передадим в коллекцию cars наш массив:

In [ ]:
db.cars.insert_many(cars_dict)

Существует также метод insert_one, если вы хотите передать в коллекцию только один элемент из массива.

Теперь посмотрим все коллекции в базе данных с именем testdb:

In [ ]:
print(db.list_collection_names())

`cars = db.cars.find()` — возвращает курсор, с помощью метода next мы можем идти по каждому элементу в переменной cars:

In [ ]:
cars = db.cars.find()

С помощью метода `list` мы можем трансформировать курсор в список и работать уже со списком:

In [ ]:
from pprint import pprint

In [ ]:
pprint(list(cars))

Например, посчитать количество записей в базе данных:

In [ ]:
number_cars = len(list(db.cars.find()))
print("Всего %d машин" %number_cars)

Также мы можем работать с элементами в переменной cars как со словарём:

In [ ]:
cars = db.cars.find()
for car in cars:
    print('Автомобиль %s стоит %.2f' %(car['name'], car['price']))

In [ ]:
expensive_cars = db.cars.find({'price': {'$gt': 50000}})
for car in expensive_cars:
    print(car['name'])

В следующем примере выведем имя и id объектов коллекции:

In [ ]:
cars = db.cars.find({}, {'_id': 1, 'name':1})
for car in cars:
    print(car)

Отсортируем элементы по убыванию цены. Для этого импортируем `DESCENDING` и напишем следующее:

In [ ]:
from pymongo import DESCENDING

In [ ]:
cars = db.cars.find().sort("price", DESCENDING)
for car in cars:
    print('Автомобиль %s стоит %.2f' %(car['name'], car['price']))

В примере вычисляется сумма всех цен на автомобили. Оператор `$sum` вычисляет и возвращает сумму числовых значений. Оператор `$group` группирует входные документы по указанному выражению-идентификатору и применяет выражения-аккумуляторы, если они указаны, к каждой группе.

In [ ]:
agr = [ {'$group': {'_id': 1, 'all': { '$sum': '$price' } } } ]

print(agr)

Агрегатный метод применяет операцию агрегирования к коллекции автомобилей.

In [ ]:
val = list(db.cars.aggregate(agr))

print('Суммарная стоимость авто: %.2f' %val[0]['all'])

Давайте посчитаем сумму цен автомобилей `Audi` и `Volvo`:

In [ ]:
car_first = 'Audi'
car_second = 'Volvo'

agr = [
    { '$match': {'$or': [ { 'name': car_first }, { 'name': car_second }] }}, 
    { '$group': {'_id': 1, 'sum2cars': { '$sum': "$price" } }}
]

val = list(db.cars.aggregate(agr))

print('Суммарная стоимость %s и %s авто: %.2f' %(car_first, car_second, val[0]['sum2cars']))

Также мы можем пропускать определённое количество элементов таблицы с помощью метода `skip` и ограничивать количество выдаваемых результатов с помощью метода `limit`:

In [ ]:
cars = db.cars.find().skip(2).limit(3)
for car in cars:
    print('Автомобиль %s стоит %.2f' %(car['name'], car['price']))

Удалить коллекцию cars из базы данных testdb:

In [ ]:
db.cars.drop()

Мы рассмотрели основные команды для работы с mongodb. Более подробную информацию ищите в документации.

## Дополнительные материалы

1. [Руководство по MongoDB](https://www.mongodb.com/docs/manual/tutorial/)

# Пример с урока

## Установим `mongo` сервер и клиент

In [1]:
!apt install mongodb > log

In [24]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Запишем данные в базу

In [44]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

In [25]:
client = MongoClient('localhost:27017')

In [94]:
db = client.marketplace

In [95]:
#collection = db['users']
collection = db.users

In [42]:
doc = {
    '_id': 3243242343243,
    'name': 'Aygul',
    'age': 30,
    'marketplace_list': ['ozon','wildberries'],
    'top_marketplace': 'wildberries',
}

In [53]:
doc_list = [{
    'name': 'Ayrat',
    'age': 18,
    'marketplace_list': ['ozon'],
},
{
    '_id': 3343243,
    'name': 'Aynaz',
    'age': 20,
    'marketplace_list': ['ozon','wildberries'],
    'last_marketplace': 'wildberries',
},
{
    'name': 'Oleg',
    'age': 30,
}]

In [47]:
try:
   collection.insert_one(doc)
except DuplicateKeyError:
  print('Объект с _id:\t%s уже записан в колекцию'%doc['_id'])


Объект с _id:	3243242343243 уже записан в колекцию


In [83]:
collection.update_one({'name': 'Erik'}, {'$set': {'last_marketplace': 'yandex', 'top_marketplace': 'yandex', 'marketplace_list': ['ozon', 'yandex']}})

In [87]:
collection.update_many({}, {'$rename': {'last_marketplace': 'lst_marketplace'}})

In [89]:
collection.delete_one({'_id': 3243242343243})

In [91]:
collection.delete_many({})

In [82]:
for doc in collection.find({'$or': [{'name': 'Erik'}, {'age': 18}]}):
  pprint(doc)

{'_id': ObjectId('6335dc9a3398df0a9c234e20'),
 'age': 33,
 'marketplace_list': ['ozon', 'yandex', 'wildberries'],
 'name': 'Erik',
 'top_marketplace': 'ozon'}
{'_id': ObjectId('6335df283398df0a9c234e21'),
 'age': 33,
 'marketplace_list': ['ozon', 'yandex', 'wildberries'],
 'name': 'Erik',
 'top_marketplace': 'ozon'}
{'_id': 3243242343243,
 'age': 33,
 'marketplace_list': ['ozon', 'yandex', 'wildberries'],
 'name': 'Erik',
 'top_marketplace': 'ozon'}
{'_id': ObjectId('6335e1773398df0a9c234e22'),
 'age': 18,
 'marketplace_list': ['ozon'],
 'name': 'Ayrat'}
{'_id': ObjectId('6335e1f63398df0a9c234e24'),
 'age': 18,
 'marketplace_list': ['ozon'],
 'name': 'Ayrat'}


In [92]:
for doc in collection.find():
  pprint(doc)

In [93]:
print(client.list_database_names()) 

['admin', 'config', 'local', 'marketplace']
